# Running an MSTIS simulation

Now we will use the initial trajectories we obtained from bootstrapping to run an MSTIS simulation. This will show both how objects can be regenerated from storage and how regenerated equivalent objects can be used in place of objects that weren't stored.

Tasks covered in this notebook:
* Loading OPS objects from storage
* Ways of assigning initial trajectories to initial samples
* Setting up a path sampling simulation with various move schemes
* Visualizing trajectories while the path sampling is running

In [1]:
from __future__ import print_function
%matplotlib inline
import openpathsampling as paths
import numpy as np

In [2]:
# monkey patches
from openpathsampling.experimental.storage import monkey_patches
#paths.CallableCV.from_dict = classmethod(monkey_patches.callable_cv_from_dict)
paths.netcdfplus.FunctionPseudoAttribute.to_dict = monkey_patches.function_pseudo_attribute_to_dict
#paths.netcdfplus.FunctionPseudoAttribute.from_dict = classmethod(monkey_patches.function_pseudo_attribute_from_dict)

## Loading things from storage

First we'll reload some of the stuff we stored before. Of course, this starts with opening the file.

In [3]:
old_store = paths.Storage("./toy_mstis_1k_OPS1_py36.nc", 'r')

A lot of information can be recovered from the old storage, and so we don't have the recreate it. However, we did not save our network, so we'll have to create a new one. Since the network creates the ensembles, that means we will have to translate the trajectories from the old ensembles to new ensembles.

Loading from storage is very easy. Each store is a list. We take the 0th snapshot as a template (it doesn't actually matter which one) for the next storage we'll create. There's only one engine stored, so we take the only one.

In [4]:
template = old_store.snapshots[0]

In [5]:
engine = old_store.engines[0]

Named objects can be found in storage by using their name as a dictionary key. This allows us to load our old collective variables and states.

In [6]:
opA = old_store.cvs['opA']
opB = old_store.cvs['opB']
opC = old_store.cvs['opC']

In [7]:
stateA = old_store.volumes['A']
stateB = old_store.volumes['B']
stateC = old_store.volumes['C']

In [8]:
# we could also load the interfaces, but it takes less code to build new ones:
interfacesA = paths.VolumeInterfaceSet(opA, 0.0,[0.2, 0.3, 0.4])
interfacesB = paths.VolumeInterfaceSet(opB, 0.0,[0.2, 0.3, 0.4])
interfacesC = paths.VolumeInterfaceSet(opC, 0.0,[0.2, 0.3, 0.4])

Once again, we have everything we need to build the MSTIS network. Recall that this will create all the ensembles we need for the simulation. However, even though the ensembles are semantically the same, these are not the same objects. We'll need to deal with that later.

In [9]:
ms_outers = paths.MSOuterTISInterface.from_lambdas(
    {ifaces: 0.5 
     for ifaces in [interfacesA, interfacesB, interfacesC]}
)
mstis = paths.MSTISNetwork(
    [(stateA, interfacesA),
     (stateB, interfacesB),
     (stateC, interfacesC)],
    ms_outers=ms_outers
)

Now we need to set up real trajectories that we can use for each of these. We can start by loading the stored sampleset.

In [10]:
# load the sampleset we have saved before
old_sampleset = old_store.samplesets[0]

### About `Sample`s

The OPS object called `Sample` is used to associate a trajectory with a replica ID and an ensemble. The trajectory needs to be associated with an ensemble so we know how to get correct statistics from the many ensembles that we might be sampling simultaneously. The trajectory needs to be associated with a replica ID so that replica exchange approaches can be analyzed.



Since the ensembles in our MSTIS network are not the exact ensemble objects that we saved our samples with (they were rebuilt), we still need a way to identify which of the new ensembles to associate them with.

There are two main ways to do this. The first is to take one trajectory, and associate it with as many ensembles as possible. If your first path comes from a TPS simulation, that is the approach you'll want to take.

The second approach is better suited to our conditions here: we already have a good trajectory for each ensemble. So we just want to remap our old ensembles to new ones.

### Loading one trajectory into lots of ensembles

In [11]:
# this makes a dictionary mapping the outermost ensemble of each sampling transition 
# to a trajectory from the old_sampleset that satisfies that ensemble
trajs = {}
for ens in [t.ensembles[-1] for t in mstis.sampling_transitions]:
    trajs[ens] = [s.trajectory for s in old_sampleset if ens(s.trajectory)][0]
    
assert(len(trajs)==3) # otherwise, we have a problem

In [12]:
initial_samples = {}
for t in mstis.sampling_transitions:
    initial_samples[t] = paths.SampleSet.map_trajectory_to_ensembles(trajs[t.ensembles[-1]], t.ensembles)

In [13]:
single_trajectory_sset = paths.SampleSet.relabel_replicas_per_ensemble(initial_samples.values())

The `sanity_check` function ensures that all trajectories in the sampleset are actually in the ensemble they claim to be associated with. At this point, we should have 9 samples.

In [14]:
single_trajectory_sset.sanity_check()
assert(len(single_trajectory_sset)==9)

### Remapping old ensembles to new ensembles

If your old and new ensembles have the same string representations, then OPS has a function to help you automatically map them. As long as you create the ensembles in the same way, they'll have the same string representation. Note that if you *don't* have the same string representation, you would have to assign trajectories to ensembles by hand (which isn't that hard, but is a bit tedious).

In [15]:
sset = paths.SampleSet.translate_ensembles(old_sampleset, mstis.sampling_ensembles)

In [16]:
sset.sanity_check()
assert(len(sset)==9)

In [17]:
# tests only: this cell sets something for the online testing
# the next cell unsets it when running the notebook live
bootstrap_sset = sset
sset = single_trajectory_sset

In [18]:
#! skip
# tests don't run this, but users should!
sset = bootstrap_sset

### Setting up special ensembles

Whichever way we initially set up the `SampleSet`, at this point it only contains samples for the main sampling trajectories of each transition. Now we need to put trajectories into various auxiliary ensembles.

#### Multiple state outer ensemble

The multiple state outer ensemble is, in fact, sampled during the bootstrapping. However, it is actually sampled once for every state that shares it. It is very easy to find a trajectory that satisfies the ensemble and to load add that sample to our sampleset.

In [19]:
for outer_ens in mstis.special_ensembles['ms_outer']:
    # doesn't matter which we take, so we take the first
    traj = next(s.trajectory for s in old_sampleset if outer_ens(s.trajectory)==True)
    samp = paths.Sample(
            replica=None,
            ensemble=outer_ens,
            trajectory=traj
    )
    # now we apply it and correct for the replica ID
    sset.append_as_new_replica(samp)

In [20]:
sset.sanity_check()
assert(len(sset)==10)

#### Minus interface ensemble

The minus interface ensembles do not yet have a trajectory. We will generate them by starting with same-state trajectories (A-to-A, B-to-B, C-to-C) in each interface, and extending into the minus ensemble.

* check whether the traj is A-to-A
* extend

First we need to make sure that the trajectory in the innermost ensemble of each state also ends in that state. This is necessary so that when we extend the trajectory, it can extends into the minus ensemble.

If the trajectory isn't right, we run a shooting move on it until it is.

In [21]:
for transition in mstis.sampling_transitions:
    innermost_ensemble = transition.ensembles[0]
    shooter = None
    if not transition.stateA(sset[innermost_ensemble].trajectory[-1]):
        shooter = paths.OneWayShootingMover(ensemble=innermost_ensemble,
                                            selector=paths.UniformSelector(),
                                            engine=engine)
        pseudoscheme = paths.LockedMoveScheme(root_mover=shooter)
        pseudosim = paths.PathSampling(storage=None, 
                                       move_scheme=pseudoscheme, 
                                       sample_set=sset,
                                      )
    while not transition.stateA(sset[innermost_ensemble].trajectory[-1]):
        pseudosim.run(1)
        sset = pseudosim.sample_set

    

Now that all the innermost ensembles are safe to use for extending into a minus interface, we extend them into a minus interface:

In [22]:
minus_samples = []
for transition in mstis.sampling_transitions:
    minus_samples.append(transition.minus_ensemble.extend_sample_from_trajectories(
        sset[transition.ensembles[0]].trajectory,
        replica=-len(minus_samples)-1,
        engine=engine
    ))
sset = sset.apply_samples(minus_samples)

In [23]:
sset.sanity_check()
assert(len(sset)==13)

## Equilibration

In molecular dynamics, you need to equilibrate if you don't start with an equilibrium frame (e.g., if you start with solvent molecules on a grid, your system should equilibrate before you start taking statistics). Similarly, if you start with a set of paths which are far from the path ensemble equilibrium, you need to equilibrate. This could either be because your trajectories are not from the real dynamics (generated with metadynamics, high temperature, etc.) or because your trajectories are not representative of the path ensemble (e.g., if you put transition trajectories into all interfaces).

As with MD, running equilibration can be the same process as running the total simulation. However, in path sampling, it doesn't have to be: we can equilibrate without replica exchange moves or path reversal moves, for example. In the example below, we create a `MoveScheme` that only includes shooting movers.

In [24]:
equil_scheme = paths.OneWayShootingMoveScheme(mstis, engine=engine)

In [25]:
equilibration = paths.PathSampling(
    storage=None,
    sample_set=sset,
    move_scheme=equil_scheme
)

In [26]:
#! skip
# tests need the unequilibrated samples to ensure passing
equilibration.run(5)

Working on Monte Carlo cycle number 5
Running for 0 seconds -  0.17 seconds per step
Estimated time remaining: 0 seconds
DONE! Completed 5 Monte Carlo cycles.


In [27]:
sset = equilibration.sample_set

## Running RETIS

Now we run the full calculation. Up to here, we haven't been storing any of our results. This time, we'll start a storage object, and we'll save the network we've created. Then we'll run a new `PathSampling` calculation object.

In [28]:
# logging creates ops_output.log file with details of what the calculation is doing
#import logging.config
#logging.config.fileConfig("../resources/logging.conf", disable_existing_loggers=False)

In [29]:
from openpathsampling.experimental.storage.ops_storage import OPSStorage, ops_class_info, ops_schema
from openpathsampling.experimental.storage.sql_backend import SQLStorageBackend

backend = SQLStorageBackend("mstis.sql", mode='w')
storage = OPSStorage.from_backend(
    backend=backend,
    schema=ops_schema,
    class_info=ops_class_info
)

In [30]:
storage.save(template)

In [31]:
print(storage.summary())

File: mstis.sql
Includes tables:
* samples: 0 items
* sample_sets: 0 items
* trajectories: 0 items
* move_changes: 0 items
* steps: 0 items
* details: 0 items
* simulation_objects: 10 items
* snapshot0: 1 items



In [32]:
mstis_calc = paths.PathSampling(
    storage=storage,
    sample_set=sset,
    move_scheme=paths.DefaultScheme(mstis, engine=engine)
)
mstis_calc.save_frequency = 50

In [33]:
class_info = storage.class_info.class_info_list[-1]
find_uuids = class_info.find_uuids

In [34]:
storage.class_info.info_from_instance(template)

ClassInfo(table=snapshot0, cls=<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>, lookup_result=('174395458874316017953765140704788480022', <class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>), find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x119e865f8>)

In [35]:
find_uuids(template, [])

({'174395458874316017953765140704788572213': <openpathsampling.engines.toy.snapshot.ToySnapshot at 0x115db9cf8>},
 [<openpathsampling.engines.toy.engine.ToyEngine at 0x11935e9e8>])

In [36]:
sset[0].trajectory[0].velocities

array([[-0.76070386, -0.6172998 ]], dtype=float32)

Now everything is ready: let's run the simulation!

In [37]:
%load_ext line_profiler
from openpathsampling.experimental.storage.serialization_helpers import get_all_uuids, unique_objects, default_find_uuids, SchemaFindUUIDs
from openpathsampling.experimental.storage.class_info import SerializationSchema

In [38]:
import os
#mstis_calc.output_stream = open(os.devnull, 'w')

In [39]:
#%%prun -D mstis.pstat
#%lprun -f unique_objects 
mstis_calc.run(200)
#mstis_calc.run(1)

Working on Monte Carlo cycle number 200
Running for 20 seconds -  0.10 seconds per step
Estimated time remaining: 0 seconds
DONE! Completed 200 Monte Carlo cycles.
 
*** Profile stats marshalled to file 'mstis.pstat'. 


In [40]:
storage.cache.fixed_cache

{'174395458874316017953765140704788480022': <openpathsampling.engines.toy.engine.ToyEngine at 0x11935e9e8>,
 '174395458874316017953765140704788480016': <openpathsampling.engines.toy.topology.ToyTopology at 0x1195e4b00>,
 '11138911631133520314863233683872022592': <openpathsampling.engines.toy.integrators.LangevinBAOABIntegrator at 0x11935e9b0>,
 '11138911631133520314863233683872022606': <openpathsampling.engines.toy.pes.PES_Add at 0x1195e4c18>,
 '11138911631133520314863233683872022602': <openpathsampling.engines.toy.pes.PES_Add at 0x1195e4ba8>,
 '11138911631133520314863233683872022604': Gaussian(-0.7000000000000001, [12. 12.], [ 0.5 -0.5]),
 '11138911631133520314863233683872022598': <openpathsampling.engines.toy.pes.PES_Add at 0x1195e4ac8>,
 '11138911631133520314863233683872022600': Gaussian(-0.7000000000000001, [12. 12.], [-0.5 -0.5]),
 '11138911631133520314863233683872022594': OuterWalls([1. 1.], [0. 0.]),
 '11138911631133520314863233683872022596': Gaussian(-0.7000000000000001, [12. 1

In [41]:
from openpathsampling.experimental.storage.class_lookup import is_storage_iterable, is_storage_mappable
is_storage_iterable._true_set

{bytes,
 list,
 openpathsampling.high_level.interface_set.VolumeInterfaceSet,
 openpathsampling.pathmover.BackwardExtendMover,
 openpathsampling.pathmover.BackwardShootMover,
 openpathsampling.pathmover.ConditionalSequentialMover,
 openpathsampling.pathmover.EnsembleFilterMover,
 openpathsampling.pathmover.FinalSubtrajectorySelectMover,
 openpathsampling.pathmover.FirstSubtrajectorySelectMover,
 openpathsampling.pathmover.ForwardExtendMover,
 openpathsampling.pathmover.ForwardShootMover,
 openpathsampling.pathmover.MinusMover,
 openpathsampling.pathmover.OneWayShootingMover,
 openpathsampling.pathmover.PathReversalMover,
 openpathsampling.pathmover.PathSimulatorMover,
 openpathsampling.pathmover.RandomChoiceMover,
 openpathsampling.pathmover.ReplicaExchangeMover,
 tuple}

In [42]:
storage.class_info.class_info_list[5]

ClassInfo(table=steps, cls=<class 'openpathsampling.pathsimulators.path_simulator.MCStep'>, lookup_result=<class 'openpathsampling.pathsimulators.path_simulator.MCStep'>, find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x115c06320>)

In [43]:
storage.class_info.class_info_list

[ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=<function default_find_uuids at 0x11579cb70>),
 ClassInfo(table=samples, cls=<class 'openpathsampling.sample.Sample'>, lookup_result=<class 'openpathsampling.sample.Sample'>, find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x115c060b8>),
 ClassInfo(table=sample_sets, cls=<class 'openpathsampling.sample.SampleSet'>, lookup_result=<class 'openpathsampling.sample.SampleSet'>, find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x115c06160>),
 ClassInfo(table=trajectories, cls=<class 'openpathsampling.engines.trajectory.Trajectory'>, lookup_result=<class 'openpathsampling.engines.trajectory.Trajectory'>, find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x115c06208>),

In [44]:
find_uuids = storage.class_info.class_info_list[-1].find_uuids

In [45]:
find_uuids(template, [])

({'174395458874316017953765140704788572213': <openpathsampling.engines.toy.snapshot.ToySnapshot at 0x115db9cf8>},
 [<openpathsampling.engines.toy.engine.ToyEngine at 0x11935e9e8>])

In [46]:
n_steps = int(mstis_calc.move_scheme.n_steps_for_trials(mstis_calc.move_scheme.movers['shooting'][0], 1000))
print(n_steps)

20100


In [47]:
storage.close()

In [48]:
backend = SQLStorageBackend("mstis.sql", mode='r')

In [49]:
list(backend.table_iterator('tables'))

[('samples', 0, 'openpathsampling.sample', 'Sample'),
 ('sample_sets', 1, 'openpathsampling.sample', 'SampleSet'),
 ('trajectories', 2, 'openpathsampling.engines.trajectory', 'Trajectory'),
 ('move_changes', 3, 'openpathsampling.movechange', 'MoveChange'),
 ('steps', 4, 'openpathsampling.pathsimulators.path_simulator', 'MCStep'),
 ('details', 5, 'openpathsampling.pathmover', 'Details'),
 ('simulation_objects', 6, 'openpathsampling.netcdfplus.base', 'StorableObject'),
 ('snapshot0', 7, 'openpathsampling.engines.toy.snapshot', 'ToySnapshot')]

In [50]:
list(backend.table_iterator('trajectories'))[-6].snapshots

'["UUID(11138911631133520314863233683872364795)", "UUID(11138911631133520314863233683872364777)", "UUID(11138911631133520314863233683872364759)", "UUID(11138911631133520314863233683872364741)", "UUID(11138911631133520314863233683872364723)", "UUID(11138911631133520314863233683872364705)", "UUID(11138911631133520314863233683872364687)", "UUID(11138911631133520314863233683872364669)", "UUID(11138911631133520314863233683872364651)", "UUID(11138911631133520314863233683872364633)", "UUID(11138911631133520314863233683872364615)", "UUID(11138911631133520314863233683872364597)", "UUID(11138911631133520314863233683872364579)", "UUID(11138911631133520314863233683872364561)", "UUID(11138911631133520314863233683872364543)", "UUID(11138911631133520314863233683872364525)", "UUID(11138911631133520314863233683872364507)", "UUID(11138911631133520314863233683872364489)", "UUID(11138911631133520314863233683872364471)", "UUID(11138911631133520314863233683872364453)", "UUID(11138911631133520314863233683872

In [51]:
import ujson as json
len(set(sum([json.loads(row.snapshots) for row in backend.table_iterator('trajectories')], [])))


7717

In [52]:
storage.class_info.special_lookup_object.is_special._true_set

{openpathsampling.engines.toy.snapshot.ToySnapshot,
 openpathsampling.movechange.AcceptedSampleMoveChange,
 openpathsampling.movechange.ConditionalSequentialMoveChange,
 openpathsampling.movechange.FilterByEnsembleMoveChange,
 openpathsampling.movechange.PathSimulatorMoveChange,
 openpathsampling.movechange.RandomChoiceMoveChange,
 openpathsampling.movechange.RejectedSampleMoveChange,
 openpathsampling.movechange.SubMoveChange,
 openpathsampling.pathmover.Details}

In [53]:
storage.class_info.special_lookup_object.is_special._false_set

{openpathsampling.collectivevariable.CoordinateFunctionCV,
 openpathsampling.engines.toy.engine.ToyEngine,
 openpathsampling.engines.toy.integrators.LangevinBAOABIntegrator,
 openpathsampling.engines.toy.pes.Gaussian,
 openpathsampling.engines.toy.pes.OuterWalls,
 openpathsampling.engines.toy.pes.PES_Add,
 openpathsampling.engines.toy.topology.ToyTopology,
 openpathsampling.engines.trajectory.Trajectory,
 openpathsampling.ensemble.AllInXEnsemble,
 openpathsampling.ensemble.AllOutXEnsemble,
 openpathsampling.ensemble.IntersectionEnsemble,
 openpathsampling.ensemble.LengthEnsemble,
 openpathsampling.ensemble.MinusInterfaceEnsemble,
 openpathsampling.ensemble.OptionalEnsemble,
 openpathsampling.ensemble.PartOutXEnsemble,
 openpathsampling.ensemble.SequentialEnsemble,
 openpathsampling.ensemble.SingleFrameEnsemble,
 openpathsampling.ensemble.TISEnsemble,
 openpathsampling.ensemble.UnionEnsemble,
 openpathsampling.high_level.interface_set.VolumeInterfaceSet,
 openpathsampling.high_level.mov

In [54]:
print(storage.summary())

File: mstis.sql
Includes tables:
* samples: 276 items
* sample_sets: 201 items
* trajectories: 166 items
* move_changes: 923 items
* steps: 201 items
* details: 914 items
* simulation_objects: 389 items
* snapshot0: 7718 items



In [55]:
mstis.to_dict()

{'from_state': {<openpathsampling.volume.CVDefinedVolume at 0x1195e4048>: <openpathsampling.high_level.transition.TISTransition at 0x1195fad68>,
  <openpathsampling.volume.CVDefinedVolume at 0x1195e4c88>: <openpathsampling.high_level.transition.TISTransition at 0x119615128>,
  <openpathsampling.volume.CVDefinedVolume at 0x1195e44e0>: <openpathsampling.high_level.transition.TISTransition at 0x11992ce80>},
 'states': (<openpathsampling.volume.CVDefinedVolume at 0x1195e4048>,
  <openpathsampling.volume.CVDefinedVolume at 0x1195e44e0>),
 'special_ensembles': {'minus': {<openpathsampling.ensemble.MinusInterfaceEnsemble at 0x119609e48>: [<openpathsampling.high_level.transition.TISTransition at 0x1195fad68>],
   <openpathsampling.ensemble.MinusInterfaceEnsemble at 0x11992c860>: [<openpathsampling.high_level.transition.TISTransition at 0x119615128>],
   <openpathsampling.ensemble.MinusInterfaceEnsemble at 0x119949588>: [<openpathsampling.high_level.transition.TISTransition at 0x11992ce80>]},
 